# Model (Random Forest)

## Create response variable

In [31]:
x1_train <- 
  sim_data_smooth %>% filter(dataset == "train") %>%
  select(id, x1) %>%
  distinct() %>% 
  pull(x1)

In [32]:
head(x1_train)

[1] 0.6066050 0.5118087 0.9837245 0.8713568 0.7376236 0.7715580

## Create data frame with PCs and response for random forest

In [33]:
rf_jfpca_df <- 
  train_transformed_jfpca$fpca_res$coef %>%
  data.frame() %>%
  rename_all(.funs = function(x) stringr::str_replace(x, "X", "pc")) %>%
  mutate(x1 = x1_train) %>%
  select(x1, everything())

In [34]:
cat(dim(train_transformed_jfpca$fpca_res$coef), "\n")

75 75 


## Fit random forest

In [35]:
set.seed(36)
rf_jfpca = randomForest(x1 ~ ., data = rf_jfpca_df)

In [61]:
rf_jfpca$ntree

[1] 500

## Evaluation

In [36]:
x1_test <- 
  sim_data %>% 
  filter(dataset == "test") %>%
  arrange(id) %>%
  select(id, x1) %>%
  distinct() %>% 
  pull(x1)

In [37]:
cat("x1_test:", length(x1_test),
    "| test curves:", nrow(test_transformed_jfpca$coef), "\n")

x1_test: 25 | test curves: 75 


In [38]:
n_rep <- nrow(test_transformed_jfpca$coef) / length(x1_test)
x1_test_expanded <- rep(x1_test, each = n_rep)

In [39]:
rf_test_df <- test_transformed_jfpca$coef %>%
  data.frame() %>%
  rename_all(~ stringr::str_replace(., "X", "pc")) %>%
  mutate(x1 = x1_test_expanded) %>%
  select(x1, everything())

In [40]:
# Automatically add missing columns to test
missing_cols <- setdiff(names(rf_jfpca_df), names(rf_test_df))

for (col in missing_cols) {
  rf_test_df[[col]] <- 0  
}

# Ensure correct column order (must match train data order)
rf_test_df <- rf_test_df[, names(rf_jfpca_df), drop = FALSE]

# ✅ Predict
pred_test <- predict(rf_jfpca, newdata = rf_test_df)

cat("✅ Prediction completed successfully! No missing PCs.\n")


✅ Prediction completed successfully! No missing PCs.


In [41]:
nmse_test <- mean((rf_test_df$x1 - pred_test)^2) / 
             mean((rf_test_df$x1 - mean(rf_test_df$x1))^2)
r2_test   <- 1 - mean((rf_test_df$x1 - pred_test)^2) / var(rf_test_df$x1)

cat("📈 NMSE:", round(nmse_test, 4), "\n")
cat("📈 R²:", round(r2_test, 4), "\n")

📈 NMSE: 1.7196 
📈 R²: -0.6967 
